# Обратная задача динамики

Рассмотрим обратную задачу динамики на примере двузвенного робота:

![двузвенный робот](./fig/two_dof.png)

In [ ]:
from sympy import *
t = Symbol("t")
g = Symbol("g")

Создадим свое собственное описание положения:

In [ ]:
class Position:
    def __init__(self, x, y, a):
        super(Position, self).__init__()
        self.x = x
        self.y = y
        self.a = a
        
        
    def __add__(self, other):
        c = cos(self.a)
        s = sin(self.a)
        return Position(
            self.x + c * other.x - s * other.y,
            self.y + s * other.x + c * other.y,
            self.a + other.a
        )

Тогда положения точек масс будет следущим:

In [ ]:
def bone_positions(q, l):
    p1 = Position(0, 0, q[0])
    p2 = p1 + Position(l[0], 0, q[1])
    p3 = p2 + Position(l[1], 0, 0)
    
    return [
        p1 + Position(l[0] / 2, 0, 0),
        p2 + Position(l[1] / 2, 0, 0),
        p3
    ]

Опишем кинетическую энергию:

In [ ]:
def kinetic_energy(p, m, j):
    vx = diff(p.x, t)
    vy = diff(p.y, t)
    v = sqrt(vx**2 + vy**2)
    omega = diff(p.a, t)
    return v**2 * m / 2 + omega**2 * j / 2

In [ ]:
def total_kinetic_energy(q, l, m, j):
    [p1, p2, p3] = bone_positions(q, l)
    kin1 = kinetic_energy(p1, m[0], j[0])
    kin2 = kinetic_energy(p2, m[1], j[1])
    kin3 = kinetic_energy(p3, m[2], j[2])
    return kin1 + kin2 + kin3

И потенциальную:

In [ ]:
def potential_energy(p, m):
    return p.y * g * m

In [ ]:
def total_potential_energy(q, l, m):
    [p1, p2, p3] = bone_positions(q, l)
    pot1 = potential_energy(p1, m[0])
    pot2 = potential_energy(p2, m[1])
    pot3 = potential_energy(p3, m[2])
    return pot1 + pot2 + pot3

Тогда Лагранжиан будет иметь слудующий вид:

In [ ]:
def lagrangian(q, l, m, j):
    return total_kinetic_energy(q, l, m, j) -\
        total_potential_energy(q, l, m)

In [ ]:
q1, q2 = symbols("q_1, q_2", cls=Function)
q = [q1(t), q2(t)]
l1, l2 = symbols("l_1, l_2")
l = [l1, l2]
m = symbols("m_1, m_2, m_3")
j = symbols("J_1, J_2, J_3")

In [ ]:
lagrangian(q, l, m, j).simplify()

Силы, необходимые для обеспечения $q$ запишем следующим образом:

In [ ]:
def force_calqulation(q, l, m, j):
    lgr = lagrangian(q, l, m, j)
    force1 = diff(diff(lgr, diff(q[0], t)), t) - diff(lgr, q[0])
    force2 = diff(diff(lgr, diff(q[1], t)), t) - diff(lgr, q[1])
    return [
        force1,
        force2
    ]

In [ ]:
forces = force_calqulation(q, l, m, j)

simplify(
    forces[0]
)

In [ ]:
simplify(
    forces[1]
)

Тогда, например, для удержания механизма в горизонтальном положении потребуется следующие силы:

In [ ]:
forces[0].replace(q[0], 0).replace(q[1], 0).simplify()

In [ ]:
forces[1].replace(q[0], 0).replace(q[1], 0).simplify()